# Data obtaining

In [1]:
import numpy as np 
import pandas as pd 
import requests
from bs4 import BeautifulSoup

In [2]:
response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(response, 'lxml')

# Data transforming

In [3]:
def parse_html_table(table):
        n_columns = 0
        n_rows=0
        column_names = []
        for row in table.find_all('tr'):
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    n_columns = len(td_tags)
                        
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())
    
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")
    
        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1
                    
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass
            
        return df

In [4]:
for table in soup.find_all('table', class_="wikitable sortable"):
    df = parse_html_table(table)                      

In [5]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


# Data cleaning

In [6]:
df = df[df.Borough != 'Not assigned']
df = df.replace('\n',' ', regex=True)
df= df[df['Neighbourhood\n'] != 'Not assigned']
df = df.groupby(['Postcode','Borough'])['Neighbourhood\n'].apply(lambda x: ", ".join(x.astype(str))).reset_index()
df = df.sample(frac=1).reset_index(drop=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M2M,North York,"Newtonbrook , Willowdale"
1,M6N,York,"The Junction North , Runnymede"
2,M4B,East York,"Woodbine Gardens , Parkview Hill"
3,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade
4,M5J,Downtown Toronto,"Harbourfront East , Toronto Islands , Union St..."
5,M8Y,Etobicoke,"Humber Bay , King's Mill Park , Kingsway Park ..."
6,M5G,Downtown Toronto,Central Bay Street
7,M3J,North York,"Northwood Park , York University"
8,M9P,Etobicoke,Westmount
9,M4R,Central Toronto,North Toronto West


In [7]:
df.shape

(103, 3)

# Geo info. obtaining

In [8]:
geo_data=pd.read_csv("http://cocl.us/Geospatial_data")
geo_data.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


# Data combining

In [9]:
geo_data.rename(columns={'Postal Code':'Postcode'}, inplace=True)
geo_data.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
full_table =df.merge(geo_data, on="Postcode", how='left')
full_table.head(20)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M2M,North York,"Newtonbrook , Willowdale",43.789053,-79.408493
1,M6N,York,"The Junction North , Runnymede",43.673185,-79.487262
2,M4B,East York,"Woodbine Gardens , Parkview Hill",43.706397,-79.309937
3,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846
4,M5J,Downtown Toronto,"Harbourfront East , Toronto Islands , Union St...",43.640816,-79.381752
5,M8Y,Etobicoke,"Humber Bay , King's Mill Park , Kingsway Park ...",43.636258,-79.498509
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M3J,North York,"Northwood Park , York University",43.767980,-79.487262
8,M9P,Etobicoke,Westmount,43.696319,-79.532242
9,M4R,Central Toronto,North Toronto West,43.715383,-79.405678


# visualize my neighborhoods in New York City

In [11]:
pip install geopy


The following command must be run outside of the IPython shell:

    $ pip install geopy

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more informations on how to install packages:

    https://docs.python.org/3/installing/


In [12]:
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
address = 'New York City'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  import sys


The geograpical coordinate of New York City are 40.7308619, -73.9871558.


In [13]:
map_geo = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(full_table['Latitude'], full_table['Longitude'], full_table['Neighbourhood\n']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_geo)  
    
map_geo